In [ ]:
from src.models import BoundingBox, Vertex
from src.services import DataConverterService, ImageDisplayService, LineDetectionService

from config import config

"""
    getting a single datapoint from the dataset 
"""

# use this when you want to load an image from the dataset whose path come frome the config files.
# converter_service = DataConverterService() 
# datapoint = converter_service.load_single_datapoint(str(0))

In [ ]:
from src.services.predict_symbols_service import PredictSymbolsService
from src.models.symbol import Symbol
from src.utils.convert_points_to_bounding_box import convert_points_to_bounding_box


"""
    predict the symbol bounding boxes using the prediction service.
"""

image_path_to_test = "D:/upload/pid/d2cf519b-b86d-42b5-9cdb-8367a2324fc4.jpg"

predicted_symbol_results = []

predict_service = PredictSymbolsService(
    image_path=image_path_to_test,
    model_path="./yolo-model-pid.pt"
)

prediction_results = predict_service.predict_bounding_boxes()

for index, pr in enumerate(prediction_results):
    symbol = Symbol(
        name=f"s-{str(index)}",
        label=pr[1],
        pointSrc=Vertex(x=pr[0][0], y=pr[0][1]),
        pointDest=Vertex(x=pr[0][2], y=pr[0][3])
    )

    predicted_symbol_results.append(symbol)

In [ ]:


"""
    predict the word bounding boxes using the azure document inteligence service.
"""

from src.services.predict_word_service import PredictWordService

word_bboxes = []
predict_word_service = PredictWordService(
    image_path=image_path_to_test
)

result = predict_word_service.predicit_bounding_boxes()

for index, item in enumerate(result): 
    word_bboxes.append(
        BoundingBox(
            name=f"w-{index}",
            pointSrc=Vertex(x=item[0],y=item[1]),
            pointDest=Vertex(x=item[2],y=item[3])
        )
    )

In [ ]:
display_service = ImageDisplayService(
    image_path_to_test,
    [*predicted_symbol_results, *word_bboxes]
)
display_service.display_image_with_bbox(color='red', dpi=100)

In [ ]:
from src.utils.calculate_distance_between_rectangles import calculate_distance_between_rectangles

"""
    remove redundent bounding boxes.
"""

index_to_delete = []

for index_1, s1 in enumerate(predicted_symbol_results):
    for index_2, s2 in enumerate(predicted_symbol_results[index_1 + 1:]):
        if(
            calculate_distance_between_rectangles(
                [*s1.pointSrc.get_dimensions(), *s1.pointDest.get_dimensions()], 
                [*s2.pointSrc.get_dimensions(), *s2.pointDest.get_dimensions()]                
            ) < 5
        ):
            index_to_delete.append(index_1)


predicted_symbol_results = [value for i, value in enumerate(predicted_symbol_results) if i not in list(set(index_to_delete))]

In [ ]:
# line detection and their extension from the datapoint
line_detection_service = LineDetectionService(
    image_path=image_path_to_test,
    bounding_boxes=[*predicted_symbol_results, *word_bboxes],
)

line_segments = [convert_points_to_bounding_box(l) for l in line_detection_service.extend_lines(
    line_detection_service.detect_line_segments(enable_thining=True)
)]

for index, l in enumerate(line_segments):
    l.name = f"l-{str(index)}"

In [ ]:
# filter the smaller lines, make them bigger than ususal. Implement them in the line service afterwards.

from src.utils.calculate_distance_between_points import calculate_distance_between_points

short_line_semgents = list(filter(lambda l: l.name in ['l-0', 'l-10'], line_segments))

print(short_line_semgents)
[calculate_distance_between_points(s.pointSrc.get_dimensions(), s.pointDest.get_dimensions()) for s in short_line_semgents]

In [ ]:
line_display_service = ImageDisplayService(
    image_path_to_test,
    [*line_segments, *predicted_symbol_results]
)

line_display_service.display_image_with_bbox(color='red', dpi=300, show_text=True)

In [ ]:
from src.services.graph_construction_service import GraphConstructionService

graph_service = GraphConstructionService(predicted_symbol_results, line_segments)
graph_service.initialize_graph()
graph_service.define_graph_edges()

In [ ]:
service = ImageDisplayService()
service.display_graph(graph_service.graph)

In [ ]:
graph_service.reduce_line_cycles()
# graph_service.set_largest_graph_connected_nodes()

In [ ]:
for i in range(100):
    graph_service.remove_zero_or_single_connection_line_nodes()

In [ ]:
graph_service.prune_multiple_path_nodes(graph_service.find_valid_paths())

In [ ]:
service.display_graph(graph_service.graph)

In [ ]:
from matplotlib import pyplot as plt
import networkx as nx

# test a graphml based graph and check if it is accurate 
r_graph = nx.parse_graphml(
  """
    <graphml xmlns="http://graphml.graphdrawing.org/xmlns" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd">
  <key id="d1" for="node" attr.name="label" attr.type="string" />
  <key id="d0" for="node" attr.name="type" attr.type="string" />
  <graph edgedefault="undirected">
    <node id="l-0">
      <data key="d0">line</data>
    </node>
    <node id="l-2">
      <data key="d0">line</data>
    </node>
    <node id="l-3">
      <data key="d0">line</data>
    </node>
    <node id="l-4">
      <data key="d0">line</data>
    </node>
    <node id="l-5">
      <data key="d0">line</data>
    </node>
    <node id="l-6">
      <data key="d0">line</data>
    </node>
    <node id="l-8">
      <data key="d0">line</data>
    </node>
    <node id="l-9">
      <data key="d0">line</data>
    </node>
    <node id="l-10">
      <data key="d0">line</data>
    </node>
    <node id="l-12">
      <data key="d0">line</data>
    </node>
    <node id="l-13">
      <data key="d0">line</data>
    </node>
    <node id="l-14">
      <data key="d0">line</data>
    </node>
    <node id="l-15">
      <data key="d0">line</data>
    </node>
    <node id="l-17">
      <data key="d0">line</data>
    </node>
    <node id="l-19">
      <data key="d0">line</data>
    </node>
    <node id="l-21">
      <data key="d0">line</data>
    </node>
    <node id="l-22">
      <data key="d0">line</data>
    </node>
    <node id="l-23">
      <data key="d0">line</data>
    </node>
    <node id="l-24">
      <data key="d0">line</data>
    </node>
    <node id="l-25">
      <data key="d0">line</data>
    </node>
    <node id="l-26">
      <data key="d0">line</data>
    </node>
    <node id="l-27">
      <data key="d0">line</data>
    </node>
    <node id="l-28">
      <data key="d0">line</data>
    </node>
    <node id="l-29">
      <data key="d0">line</data>
    </node>
    <node id="l-30">
      <data key="d0">line</data>
    </node>
    <node id="l-31">
      <data key="d0">line</data>
    </node>
    <node id="l-33">
      <data key="d0">line</data>
    </node>
    <node id="l-35">
      <data key="d0">line</data>
    </node>
    <node id="l-36">
      <data key="d0">line</data>
    </node>
    <node id="l-39">
      <data key="d0">line</data>
    </node>
    <node id="l-40">
      <data key="d0">line</data>
    </node>
    <node id="l-41">
      <data key="d0">line</data>
    </node>
    <node id="l-46">
      <data key="d0">line</data>
    </node>
    <node id="l-48">
      <data key="d0">line</data>
    </node>
    <node id="l-50">
      <data key="d0">line</data>
    </node>
    <node id="l-54">
      <data key="d0">line</data>
    </node>
    <node id="l-56">
      <data key="d0">line</data>
    </node>
    <node id="l-58">
      <data key="d0">line</data>
    </node>
    <node id="l-64">
      <data key="d0">line</data>
    </node>
    <node id="l-65">
      <data key="d0">line</data>
    </node>
    <node id="l-69">
      <data key="d0">line</data>
    </node>
    <node id="l-71">
      <data key="d0">line</data>
    </node>
    <node id="l-75">
      <data key="d0">line</data>
    </node>
    <node id="l-78">
      <data key="d0">line</data>
    </node>
    <node id="l-85">
      <data key="d0">line</data>
    </node>
    <node id="l-89">
      <data key="d0">line</data>
    </node>
    <node id="l-100">
      <data key="d0">line</data>
    </node>
    <node id="l-102">
      <data key="d0">line</data>
    </node>
    <node id="l-104">
      <data key="d0">line</data>
    </node>
    <node id="l-105">
      <data key="d0">line</data>
    </node>
    <node id="l-110">
      <data key="d0">line</data>
    </node>
    <node id="l-111">
      <data key="d0">line</data>
    </node>
    <node id="l-114">
      <data key="d0">line</data>
    </node>
    <node id="l-116">
      <data key="d0">line</data>
    </node>
    <node id="l-118">
      <data key="d0">line</data>
    </node>
    <node id="l-125">
      <data key="d0">line</data>
    </node>
    <node id="l-128">
      <data key="d0">line</data>
    </node>
    <node id="l-133">
      <data key="d0">line</data>
    </node>
    <node id="l-135">
      <data key="d0">line</data>
    </node>
    <node id="l-136">
      <data key="d0">line</data>
    </node>
    <node id="l-139">
      <data key="d0">line</data>
    </node>
    <node id="l-143">
      <data key="d0">line</data>
    </node>
    <node id="l-145">
      <data key="d0">line</data>
    </node>
    <node id="l-148">
      <data key="d0">line</data>
    </node>
    <node id="l-152">
      <data key="d0">line</data>
    </node>
    <node id="l-156">
      <data key="d0">line</data>
    </node>
    <node id="l-160">
      <data key="d0">line</data>
    </node>
    <node id="l-163">
      <data key="d0">line</data>
    </node>
    <node id="l-168">
      <data key="d0">line</data>
    </node>
    <node id="l-170">
      <data key="d0">line</data>
    </node>
    <node id="l-172">
      <data key="d0">line</data>
    </node>
    <node id="l-175">
      <data key="d0">line</data>
    </node>
    <node id="l-183">
      <data key="d0">line</data>
    </node>
    <node id="l-190">
      <data key="d0">line</data>
    </node>
    <node id="l-192">
      <data key="d0">line</data>
    </node>
    <node id="l-195">
      <data key="d0">line</data>
    </node>
    <node id="l-196">
      <data key="d0">line</data>
    </node>
    <node id="l-198">
      <data key="d0">line</data>
    </node>
    <node id="l-200">
      <data key="d0">line</data>
    </node>
    <node id="l-206">
      <data key="d0">line</data>
    </node>
    <node id="l-211">
      <data key="d0">line</data>
    </node>
    <node id="l-215">
      <data key="d0">line</data>
    </node>
    <node id="l-218">
      <data key="d0">line</data>
    </node>
    <node id="l-222">
      <data key="d0">line</data>
    </node>
    <node id="l-229">
      <data key="d0">line</data>
    </node>
    <node id="l-230">
      <data key="d0">line</data>
    </node>
    <node id="l-231">
      <data key="d0">line</data>
    </node>
    <node id="l-236">
      <data key="d0">line</data>
    </node>
    <node id="l-238">
      <data key="d0">line</data>
    </node>
    <node id="l-241">
      <data key="d0">line</data>
    </node>
    <node id="l-245">
      <data key="d0">line</data>
    </node>
    <node id="l-248">
      <data key="d0">line</data>
    </node>
    <node id="l-252">
      <data key="d0">line</data>
    </node>
    <node id="l-256">
      <data key="d0">line</data>
    </node>
    <node id="l-264">
      <data key="d0">line</data>
    </node>
    <node id="l-266">
      <data key="d0">line</data>
    </node>
    <node id="l-267">
      <data key="d0">line</data>
    </node>
    <node id="l-268">
      <data key="d0">line</data>
    </node>
    <node id="l-271">
      <data key="d0">line</data>
    </node>
    <node id="l-272">
      <data key="d0">line</data>
    </node>
    <node id="l-275">
      <data key="d0">line</data>
    </node>
    <node id="l-276">
      <data key="d0">line</data>
    </node>
    <node id="l-278">
      <data key="d0">line</data>
    </node>
    <node id="l-279">
      <data key="d0">line</data>
    </node>
    <node id="l-281">
      <data key="d0">line</data>
    </node>
    <node id="l-287">
      <data key="d0">line</data>
    </node>
    <node id="l-289">
      <data key="d0">line</data>
    </node>
    <node id="l-294">
      <data key="d0">line</data>
    </node>
    <node id="l-298">
      <data key="d0">line</data>
    </node>
    <node id="l-310">
      <data key="d0">line</data>
    </node>
    <node id="l-316">
      <data key="d0">line</data>
    </node>
    <node id="l-324">
      <data key="d0">line</data>
    </node>
    <node id="l-326">
      <data key="d0">line</data>
    </node>
    <node id="l-332">
      <data key="d0">line</data>
    </node>
    <node id="l-335">
      <data key="d0">line</data>
    </node>
    <node id="l-336">
      <data key="d0">line</data>
    </node>
    <node id="l-338">
      <data key="d0">line</data>
    </node>
    <node id="l-340">
      <data key="d0">line</data>
    </node>
    <node id="l-342">
      <data key="d0">line</data>
    </node>
    <node id="l-344">
      <data key="d0">line</data>
    </node>
    <node id="l-347">
      <data key="d0">line</data>
    </node>
    <node id="l-351">
      <data key="d0">line</data>
    </node>
    <node id="l-352">
      <data key="d0">line</data>
    </node>
    <node id="l-354">
      <data key="d0">line</data>
    </node>
    <node id="l-356">
      <data key="d0">line</data>
    </node>
    <node id="l-359">
      <data key="d0">line</data>
    </node>
    <node id="l-360">
      <data key="d0">line</data>
    </node>
    <node id="l-363">
      <data key="d0">line</data>
    </node>
    <node id="l-369">
      <data key="d0">line</data>
    </node>
    <node id="l-370">
      <data key="d0">line</data>
    </node>
    <node id="l-371">
      <data key="d0">line</data>
    </node>
    <node id="l-378">
      <data key="d0">line</data>
    </node>
    <node id="l-385">
      <data key="d0">line</data>
    </node>
    <node id="l-386">
      <data key="d0">line</data>
    </node>
    <node id="l-389">
      <data key="d0">line</data>
    </node>
    <node id="l-394">
      <data key="d0">line</data>
    </node>
    <node id="l-399">
      <data key="d0">line</data>
    </node>
    <node id="l-407">
      <data key="d0">line</data>
    </node>
    <node id="l-408">
      <data key="d0">line</data>
    </node>
    <node id="l-429">
      <data key="d0">line</data>
    </node>
    <node id="l-437">
      <data key="d0">line</data>
    </node>
    <node id="l-444">
      <data key="d0">line</data>
    </node>
    <node id="l-445">
      <data key="d0">line</data>
    </node>
    <node id="l-449">
      <data key="d0">line</data>
    </node>
    <node id="l-459">
      <data key="d0">line</data>
    </node>
    <node id="l-464">
      <data key="d0">line</data>
    </node>
    <node id="l-465">
      <data key="d0">line</data>
    </node>
    <node id="l-467">
      <data key="d0">line</data>
    </node>
    <node id="l-470">
      <data key="d0">line</data>
    </node>
    <node id="l-474">
      <data key="d0">line</data>
    </node>
    <node id="l-479">
      <data key="d0">line</data>
    </node>
    <node id="l-480">
      <data key="d0">line</data>
    </node>
    <node id="l-487">
      <data key="d0">line</data>
    </node>
    <node id="l-494">
      <data key="d0">line</data>
    </node>
    <node id="l-497">
      <data key="d0">line</data>
    </node>
    <node id="l-499">
      <data key="d0">line</data>
    </node>
    <node id="l-500">
      <data key="d0">line</data>
    </node>
    <node id="l-511">
      <data key="d0">line</data>
    </node>
    <node id="l-512">
      <data key="d0">line</data>
    </node>
    <node id="l-516">
      <data key="d0">line</data>
    </node>
    <node id="l-520">
      <data key="d0">line</data>
    </node>
    <node id="l-522">
      <data key="d0">line</data>
    </node>
    <node id="l-529">
      <data key="d0">line</data>
    </node>
    <node id="l-536">
      <data key="d0">line</data>
    </node>
    <node id="l-541">
      <data key="d0">line</data>
    </node>
    <node id="l-542">
      <data key="d0">line</data>
    </node>
    <node id="l-545">
      <data key="d0">line</data>
    </node>
    <node id="l-552">
      <data key="d0">line</data>
    </node>
    <node id="l-564">
      <data key="d0">line</data>
    </node>
    <node id="l-566">
      <data key="d0">line</data>
    </node>
    <node id="l-573">
      <data key="d0">line</data>
    </node>
    <node id="l-581">
      <data key="d0">line</data>
    </node>
    <node id="l-582">
      <data key="d0">line</data>
    </node>
    <node id="l-585">
      <data key="d0">line</data>
    </node>
    <node id="l-591">
      <data key="d0">line</data>
    </node>
    <node id="l-594">
      <data key="d0">line</data>
    </node>
    <node id="l-597">
      <data key="d0">line</data>
    </node>
    <node id="l-599">
      <data key="d0">line</data>
    </node>
    <node id="l-601">
      <data key="d0">line</data>
    </node>
    <node id="l-604">
      <data key="d0">line</data>
    </node>
    <node id="l-615">
      <data key="d0">line</data>
    </node>
    <node id="l-619">
      <data key="d0">line</data>
    </node>
    <node id="l-621">
      <data key="d0">line</data>
    </node>
    <node id="l-637">
      <data key="d0">line</data>
    </node>
    <node id="l-648">
      <data key="d0">line</data>
    </node>
    <node id="l-666">
      <data key="d0">line</data>
    </node>
    <node id="l-686">
      <data key="d0">line</data>
    </node>
    <node id="l-695">
      <data key="d0">line</data>
    </node>
    <node id="l-700">
      <data key="d0">line</data>
    </node>
    <node id="l-701">
      <data key="d0">line</data>
    </node>
    <node id="l-704">
      <data key="d0">line</data>
    </node>
    <node id="l-707">
      <data key="d0">line</data>
    </node>
    <node id="l-716">
      <data key="d0">line</data>
    </node>
    <node id="l-728">
      <data key="d0">line</data>
    </node>
    <node id="l-738">
      <data key="d0">line</data>
    </node>
    <node id="l-740">
      <data key="d0">line</data>
    </node>
    <node id="l-741">
      <data key="d0">line</data>
    </node>
    <node id="l-747">
      <data key="d0">line</data>
    </node>
    <node id="l-748">
      <data key="d0">line</data>
    </node>
    <node id="l-752">
      <data key="d0">line</data>
    </node>
    <node id="l-760">
      <data key="d0">line</data>
    </node>
    <node id="l-766">
      <data key="d0">line</data>
    </node>
    <node id="l-772">
      <data key="d0">line</data>
    </node>
    <node id="l-780">
      <data key="d0">line</data>
    </node>
    <node id="l-782">
      <data key="d0">line</data>
    </node>
    <node id="l-803">
      <data key="d0">line</data>
    </node>
    <node id="l-834">
      <data key="d0">line</data>
    </node>
    <node id="l-840">
      <data key="d0">line</data>
    </node>
    <node id="l-850">
      <data key="d0">line</data>
    </node>
    <node id="l-852">
      <data key="d0">line</data>
    </node>
    <node id="l-855">
      <data key="d0">line</data>
    </node>
    <node id="l-859">
      <data key="d0">line</data>
    </node>
    <node id="l-866">
      <data key="d0">line</data>
    </node>
    <node id="l-884">
      <data key="d0">line</data>
    </node>
    <node id="l-885">
      <data key="d0">line</data>
    </node>
    <node id="l-891">
      <data key="d0">line</data>
    </node>
    <node id="l-900">
      <data key="d0">line</data>
    </node>
    <node id="l-905">
      <data key="d0">line</data>
    </node>
    <node id="l-913">
      <data key="d0">line</data>
    </node>
    <node id="l-923">
      <data key="d0">line</data>
    </node>
    <node id="l-932">
      <data key="d0">line</data>
    </node>
    <node id="l-940">
      <data key="d0">line</data>
    </node>
    <node id="l-949">
      <data key="d0">line</data>
    </node>
    <node id="l-953">
      <data key="d0">line</data>
    </node>
    <node id="l-955">
      <data key="d0">line</data>
    </node>
    <node id="l-963">
      <data key="d0">line</data>
    </node>
    <node id="l-964">
      <data key="d0">line</data>
    </node>
    <node id="l-977">
      <data key="d0">line</data>
    </node>
    <node id="l-989">
      <data key="d0">line</data>
    </node>
    <node id="l-990">
      <data key="d0">line</data>
    </node>
    <node id="l-998">
      <data key="d0">line</data>
    </node>
    <node id="l-1003">
      <data key="d0">line</data>
    </node>
    <node id="l-1006">
      <data key="d0">line</data>
    </node>
    <node id="l-1028">
      <data key="d0">line</data>
    </node>
    <node id="l-1042">
      <data key="d0">line</data>
    </node>
    <node id="l-1046">
      <data key="d0">line</data>
    </node>
    <node id="l-1047">
      <data key="d0">line</data>
    </node>
    <node id="l-1063">
      <data key="d0">line</data>
    </node>
    <node id="l-1070">
      <data key="d0">line</data>
    </node>
    <node id="l-1071">
      <data key="d0">line</data>
    </node>
    <node id="l-1078">
      <data key="d0">line</data>
    </node>
    <node id="l-1082">
      <data key="d0">line</data>
    </node>
    <node id="l-1083">
      <data key="d0">line</data>
    </node>
    <node id="l-1092">
      <data key="d0">line</data>
    </node>
    <node id="l-1099">
      <data key="d0">line</data>
    </node>
    <node id="l-1105">
      <data key="d0">line</data>
    </node>
    <node id="l-1106">
      <data key="d0">line</data>
    </node>
    <node id="l-1108">
      <data key="d0">line</data>
    </node>
    <node id="l-1116">
      <data key="d0">line</data>
    </node>
    <node id="l-1119">
      <data key="d0">line</data>
    </node>
    <node id="l-1122">
      <data key="d0">line</data>
    </node>
    <node id="l-1123">
      <data key="d0">line</data>
    </node>
    <node id="l-1124">
      <data key="d0">line</data>
    </node>
    <node id="l-1128">
      <data key="d0">line</data>
    </node>
    <node id="l-1132">
      <data key="d0">line</data>
    </node>
    <node id="l-1136">
      <data key="d0">line</data>
    </node>
    <node id="l-1137">
      <data key="d0">line</data>
    </node>
    <node id="l-1138">
      <data key="d0">line</data>
    </node>
    <node id="l-1139">
      <data key="d0">line</data>
    </node>
    <node id="l-1141">
      <data key="d0">line</data>
    </node>
    <node id="s-0">
      <data key="d0">symbol</data>
      <data key="d1">9</data>
    </node>
    <node id="s-1">
      <data key="d0">symbol</data>
      <data key="d1">25</data>
    </node>
    <node id="s-2">
      <data key="d0">symbol</data>
      <data key="d1">11</data>
    </node>
    <node id="s-3">
      <data key="d0">symbol</data>
      <data key="d1">12</data>
    </node>
    <node id="s-4">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-5">
      <data key="d0">symbol</data>
      <data key="d1">8</data>
    </node>
    <node id="s-6">
      <data key="d0">symbol</data>
      <data key="d1">25</data>
    </node>
    <node id="s-7">
      <data key="d0">symbol</data>
      <data key="d1">24</data>
    </node>
    <node id="s-8">
      <data key="d0">symbol</data>
      <data key="d1">2</data>
    </node>
    <node id="s-9">
      <data key="d0">symbol</data>
      <data key="d1">20</data>
    </node>
    <node id="s-10">
      <data key="d0">symbol</data>
      <data key="d1">22</data>
    </node>
    <node id="s-11">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-12">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-13">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-14">
      <data key="d0">symbol</data>
      <data key="d1">22</data>
    </node>
    <node id="s-15">
      <data key="d0">symbol</data>
      <data key="d1">12</data>
    </node>
    <node id="s-16">
      <data key="d0">symbol</data>
      <data key="d1">26</data>
    </node>
    <node id="s-17">
      <data key="d0">symbol</data>
      <data key="d1">2</data>
    </node>
    <node id="s-18">
      <data key="d0">symbol</data>
      <data key="d1">22</data>
    </node>
    <node id="s-19">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-20">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-21">
      <data key="d0">symbol</data>
      <data key="d1">29</data>
    </node>
    <node id="s-22">
      <data key="d0">symbol</data>
      <data key="d1">12</data>
    </node>
    <node id="s-23">
      <data key="d0">symbol</data>
      <data key="d1">18</data>
    </node>
    <node id="s-24">
      <data key="d0">symbol</data>
      <data key="d1">20</data>
    </node>
    <node id="s-25">
      <data key="d0">symbol</data>
      <data key="d1">26</data>
    </node>
    <node id="s-26">
      <data key="d0">symbol</data>
      <data key="d1">1</data>
    </node>
    <node id="s-27">
      <data key="d0">symbol</data>
      <data key="d1">28</data>
    </node>
    <node id="s-28">
      <data key="d0">symbol</data>
      <data key="d1">12</data>
    </node>
    <node id="s-29">
      <data key="d0">symbol</data>
      <data key="d1">32</data>
    </node>
    <node id="s-30">
      <data key="d0">symbol</data>
      <data key="d1">8</data>
    </node>
    <node id="s-31">
      <data key="d0">symbol</data>
      <data key="d1">25</data>
    </node>
    <node id="s-32">
      <data key="d0">symbol</data>
      <data key="d1">5</data>
    </node>
    <node id="s-33">
      <data key="d0">symbol</data>
      <data key="d1">22</data>
    </node>
    <node id="s-34">
      <data key="d0">symbol</data>
      <data key="d1">23</data>
    </node>
    <node id="s-35">
      <data key="d0">symbol</data>
      <data key="d1">7</data>
    </node>
    <node id="s-36">
      <data key="d0">symbol</data>
      <data key="d1">8</data>
    </node>
    <node id="s-37">
      <data key="d0">symbol</data>
      <data key="d1">7</data>
    </node>
    <node id="s-38">
      <data key="d0">symbol</data>
      <data key="d1">9</data>
    </node>
    <node id="s-39">
      <data key="d0">symbol</data>
      <data key="d1">7</data>
    </node>
    <node id="s-40">
      <data key="d0">symbol</data>
      <data key="d1">17</data>
    </node>
    <node id="s-41">
      <data key="d0">symbol</data>
      <data key="d1">4</data>
    </node>
    <node id="s-42">
      <data key="d0">symbol</data>
      <data key="d1">18</data>
    </node>
    <node id="s-43">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-44">
      <data key="d0">symbol</data>
      <data key="d1">20</data>
    </node>
    <node id="s-45">
      <data key="d0">symbol</data>
      <data key="d1">32</data>
    </node>
    <node id="s-46">
      <data key="d0">symbol</data>
      <data key="d1">25</data>
    </node>
    <node id="s-47">
      <data key="d0">symbol</data>
      <data key="d1">7</data>
    </node>
    <node id="s-48">
      <data key="d0">symbol</data>
      <data key="d1">18</data>
    </node>
    <node id="s-49">
      <data key="d0">symbol</data>
      <data key="d1">15</data>
    </node>
    <node id="s-50">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-51">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-52">
      <data key="d0">symbol</data>
      <data key="d1">16</data>
    </node>
    <node id="s-53">
      <data key="d0">symbol</data>
      <data key="d1">31</data>
    </node>
    <node id="s-54">
      <data key="d0">symbol</data>
      <data key="d1">30</data>
    </node>
    <node id="s-55">
      <data key="d0">symbol</data>
      <data key="d1">7</data>
    </node>
    <node id="s-56">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-57">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-58">
      <data key="d0">symbol</data>
      <data key="d1">2</data>
    </node>
    <node id="s-59">
      <data key="d0">symbol</data>
      <data key="d1">8</data>
    </node>
    <node id="s-60">
      <data key="d0">symbol</data>
      <data key="d1">23</data>
    </node>
    <node id="s-61">
      <data key="d0">symbol</data>
      <data key="d1">32</data>
    </node>
    <node id="s-62">
      <data key="d0">symbol</data>
      <data key="d1">20</data>
    </node>
    <node id="s-63">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-64">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-65">
      <data key="d0">symbol</data>
      <data key="d1">6</data>
    </node>
    <node id="s-66">
      <data key="d0">symbol</data>
      <data key="d1">6</data>
    </node>
    <node id="s-67">
      <data key="d0">symbol</data>
      <data key="d1">30</data>
    </node>
    <node id="s-68">
      <data key="d0">symbol</data>
      <data key="d1">4</data>
    </node>
    <node id="s-69">
      <data key="d0">symbol</data>
      <data key="d1">23</data>
    </node>
    <node id="s-70">
      <data key="d0">symbol</data>
      <data key="d1">19</data>
    </node>
    <node id="s-71">
      <data key="d0">symbol</data>
      <data key="d1">2</data>
    </node>
    <node id="s-72">
      <data key="d0">symbol</data>
      <data key="d1">18</data>
    </node>
    <node id="s-73">
      <data key="d0">symbol</data>
      <data key="d1">29</data>
    </node>
    <node id="s-74">
      <data key="d0">symbol</data>
      <data key="d1">29</data>
    </node>
    <node id="s-75">
      <data key="d0">symbol</data>
      <data key="d1">13</data>
    </node>
    <node id="s-76">
      <data key="d0">symbol</data>
      <data key="d1">27</data>
    </node>
    <node id="s-77">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-78">
      <data key="d0">symbol</data>
      <data key="d1">28</data>
    </node>
    <node id="s-79">
      <data key="d0">symbol</data>
      <data key="d1">17</data>
    </node>
    <node id="s-80">
      <data key="d0">symbol</data>
      <data key="d1">1</data>
    </node>
    <node id="s-81">
      <data key="d0">symbol</data>
      <data key="d1">20</data>
    </node>
    <node id="s-82">
      <data key="d0">symbol</data>
      <data key="d1">6</data>
    </node>
    <node id="s-83">
      <data key="d0">symbol</data>
      <data key="d1">28</data>
    </node>
    <node id="s-84">
      <data key="d0">symbol</data>
      <data key="d1">10</data>
    </node>
    <node id="s-85">
      <data key="d0">symbol</data>
      <data key="d1">13</data>
    </node>
    <node id="s-86">
      <data key="d0">symbol</data>
      <data key="d1">31</data>
    </node>
    <node id="s-87">
      <data key="d0">symbol</data>
      <data key="d1">4</data>
    </node>
    <node id="s-88">
      <data key="d0">symbol</data>
      <data key="d1">21</data>
    </node>
    <node id="s-89">
      <data key="d0">symbol</data>
      <data key="d1">8</data>
    </node>
    <node id="p-0">
      <data key="d0">connector</data>
    </node>
    <node id="p-1">
      <data key="d0">connector</data>
    </node>
    <node id="p-2">
      <data key="d0">connector</data>
    </node>
    <node id="p-3">
      <data key="d0">connector</data>
    </node>
    <node id="p-4">
      <data key="d0">connector</data>
    </node>
    <node id="p-5">
      <data key="d0">connector</data>
    </node>
    <node id="p-6">
      <data key="d0">connector</data>
    </node>
    <node id="p-7">
      <data key="d0">connector</data>
    </node>
    <node id="p-8">
      <data key="d0">connector</data>
    </node>
    <node id="p-9">
      <data key="d0">connector</data>
    </node>
    <node id="p-10">
      <data key="d0">connector</data>
    </node>
    <node id="p-11">
      <data key="d0">connector</data>
    </node>
    <node id="p-12">
      <data key="d0">connector</data>
    </node>
    <node id="p-13">
      <data key="d0">connector</data>
    </node>
    <node id="p-14">
      <data key="d0">connector</data>
    </node>
    <node id="p-15">
      <data key="d0">connector</data>
    </node>
    <node id="p-16">
      <data key="d0">connector</data>
    </node>
    <node id="p-17">
      <data key="d0">connector</data>
    </node>
    <node id="p-18">
      <data key="d0">connector</data>
    </node>
    <node id="p-19">
      <data key="d0">connector</data>
    </node>
    <node id="p-20">
      <data key="d0">connector</data>
    </node>
    <node id="p-21">
      <data key="d0">connector</data>
    </node>
    <node id="p-22">
      <data key="d0">connector</data>
    </node>
    <node id="p-23">
      <data key="d0">connector</data>
    </node>
    <node id="p-24">
      <data key="d0">connector</data>
    </node>
    <node id="p-25">
      <data key="d0">connector</data>
    </node>
    <node id="p-26">
      <data key="d0">connector</data>
    </node>
    <node id="p-27">
      <data key="d0">connector</data>
    </node>
    <node id="p-28">
      <data key="d0">connector</data>
    </node>
    <node id="p-29">
      <data key="d0">connector</data>
    </node>
    <node id="p-30">
      <data key="d0">connector</data>
    </node>
    <node id="p-31">
      <data key="d0">connector</data>
    </node>
    <node id="p-32">
      <data key="d0">connector</data>
    </node>
    <node id="p-33">
      <data key="d0">connector</data>
    </node>
    <node id="p-34">
      <data key="d0">connector</data>
    </node>
    <node id="p-35">
      <data key="d0">connector</data>
    </node>
    <node id="p-36">
      <data key="d0">connector</data>
    </node>
    <node id="p-37">
      <data key="d0">connector</data>
    </node>
    <node id="p-38">
      <data key="d0">connector</data>
    </node>
    <node id="p-39">
      <data key="d0">connector</data>
    </node>
    <node id="p-40">
      <data key="d0">connector</data>
    </node>
    <node id="p-41">
      <data key="d0">connector</data>
    </node>
    <node id="p-42">
      <data key="d0">connector</data>
    </node>
    <node id="p-43">
      <data key="d0">connector</data>
    </node>
    <node id="p-44">
      <data key="d0">connector</data>
    </node>
    <node id="p-45">
      <data key="d0">connector</data>
    </node>
    <node id="p-46">
      <data key="d0">connector</data>
    </node>
    <node id="p-47">
      <data key="d0">connector</data>
    </node>
    <node id="p-48">
      <data key="d0">connector</data>
    </node>
    <node id="p-49">
      <data key="d0">connector</data>
    </node>
    <node id="p-50">
      <data key="d0">connector</data>
    </node>
    <node id="p-51">
      <data key="d0">connector</data>
    </node>
    <node id="p-52">
      <data key="d0">connector</data>
    </node>
    <node id="p-53">
      <data key="d0">connector</data>
    </node>
    <node id="p-54">
      <data key="d0">connector</data>
    </node>
    <node id="p-55">
      <data key="d0">connector</data>
    </node>
    <node id="p-56">
      <data key="d0">connector</data>
    </node>
    <node id="p-57">
      <data key="d0">connector</data>
    </node>
    <node id="p-58">
      <data key="d0">connector</data>
    </node>
    <node id="p-59">
      <data key="d0">connector</data>
    </node>
    <node id="p-60">
      <data key="d0">connector</data>
    </node>
    <node id="p-61">
      <data key="d0">connector</data>
    </node>
    <node id="p-62">
      <data key="d0">connector</data>
    </node>
    <node id="p-63">
      <data key="d0">connector</data>
    </node>
    <node id="p-64">
      <data key="d0">connector</data>
    </node>
    <node id="p-65">
      <data key="d0">connector</data>
    </node>
    <node id="p-66">
      <data key="d0">connector</data>
    </node>
    <node id="p-67">
      <data key="d0">connector</data>
    </node>
    <node id="p-68">
      <data key="d0">connector</data>
    </node>
    <node id="p-69">
      <data key="d0">connector</data>
    </node>
    <node id="p-70">
      <data key="d0">connector</data>
    </node>
    <node id="p-71">
      <data key="d0">connector</data>
    </node>
    <node id="p-72">
      <data key="d0">connector</data>
    </node>
    <node id="p-73">
      <data key="d0">connector</data>
    </node>
    <node id="p-74">
      <data key="d0">connector</data>
    </node>
    <node id="p-75">
      <data key="d0">connector</data>
    </node>
    <node id="p-76">
      <data key="d0">connector</data>
    </node>
    <node id="p-77">
      <data key="d0">connector</data>
    </node>
    <node id="p-78">
      <data key="d0">connector</data>
    </node>
    <node id="p-79">
      <data key="d0">connector</data>
    </node>
    <node id="p-80">
      <data key="d0">connector</data>
    </node>
    <node id="p-81">
      <data key="d0">connector</data>
    </node>
    <node id="p-82">
      <data key="d0">connector</data>
    </node>
    <node id="p-83">
      <data key="d0">connector</data>
    </node>
    <node id="p-84">
      <data key="d0">connector</data>
    </node>
    <node id="p-85">
      <data key="d0">connector</data>
    </node>
    <node id="p-86">
      <data key="d0">connector</data>
    </node>
    <node id="p-87">
      <data key="d0">connector</data>
    </node>
    <node id="p-88">
      <data key="d0">connector</data>
    </node>
    <node id="p-89">
      <data key="d0">connector</data>
    </node>
    <node id="p-90">
      <data key="d0">connector</data>
    </node>
    <node id="p-91">
      <data key="d0">connector</data>
    </node>
    <node id="p-92">
      <data key="d0">connector</data>
    </node>
    <node id="p-93">
      <data key="d0">connector</data>
    </node>
    <node id="p-94">
      <data key="d0">connector</data>
    </node>
    <node id="p-95">
      <data key="d0">connector</data>
    </node>
    <node id="p-96">
      <data key="d0">connector</data>
    </node>
    <node id="p-97">
      <data key="d0">connector</data>
    </node>
    <node id="p-98">
      <data key="d0">connector</data>
    </node>
    <node id="p-99">
      <data key="d0">connector</data>
    </node>
    <node id="p-100">
      <data key="d0">connector</data>
    </node>
    <node id="p-101">
      <data key="d0">connector</data>
    </node>
    <node id="p-102">
      <data key="d0">connector</data>
    </node>
    <node id="p-103">
      <data key="d0">connector</data>
    </node>
    <node id="p-104">
      <data key="d0">connector</data>
    </node>
    <node id="p-105">
      <data key="d0">connector</data>
    </node>
    <node id="p-106">
      <data key="d0">connector</data>
    </node>
    <node id="p-107">
      <data key="d0">connector</data>
    </node>
    <node id="p-108">
      <data key="d0">connector</data>
    </node>
    <node id="p-109">
      <data key="d0">connector</data>
    </node>
    <node id="p-110">
      <data key="d0">connector</data>
    </node>
    <node id="p-111">
      <data key="d0">connector</data>
    </node>
    <node id="p-112">
      <data key="d0">connector</data>
    </node>
    <node id="p-113">
      <data key="d0">connector</data>
    </node>
    <node id="p-114">
      <data key="d0">connector</data>
    </node>
    <node id="p-115">
      <data key="d0">connector</data>
    </node>
    <node id="p-116">
      <data key="d0">connector</data>
    </node>
    <node id="p-117">
      <data key="d0">connector</data>
    </node>
    <node id="p-118">
      <data key="d0">connector</data>
    </node>
    <node id="p-119">
      <data key="d0">connector</data>
    </node>
    <node id="p-120">
      <data key="d0">connector</data>
    </node>
    <node id="p-121">
      <data key="d0">connector</data>
    </node>
    <node id="p-122">
      <data key="d0">connector</data>
    </node>
    <node id="p-123">
      <data key="d0">connector</data>
    </node>
    <node id="p-124">
      <data key="d0">connector</data>
    </node>
    <node id="p-125">
      <data key="d0">connector</data>
    </node>
    <node id="p-126">
      <data key="d0">connector</data>
    </node>
    <node id="p-127">
      <data key="d0">connector</data>
    </node>
    <node id="p-128">
      <data key="d0">connector</data>
    </node>
    <node id="p-129">
      <data key="d0">connector</data>
    </node>
    <node id="p-130">
      <data key="d0">connector</data>
    </node>
    <node id="p-131">
      <data key="d0">connector</data>
    </node>
    <node id="p-132">
      <data key="d0">connector</data>
    </node>
    <node id="p-133">
      <data key="d0">connector</data>
    </node>
    <node id="p-134">
      <data key="d0">connector</data>
    </node>
    <node id="p-135">
      <data key="d0">connector</data>
    </node>
    <node id="p-136">
      <data key="d0">connector</data>
    </node>
    <node id="p-137">
      <data key="d0">connector</data>
    </node>
    <node id="p-138">
      <data key="d0">connector</data>
    </node>
    <node id="p-139">
      <data key="d0">connector</data>
    </node>
    <node id="p-140">
      <data key="d0">connector</data>
    </node>
    <node id="p-141">
      <data key="d0">connector</data>
    </node>
    <node id="p-142">
      <data key="d0">connector</data>
    </node>
    <node id="p-143">
      <data key="d0">connector</data>
    </node>
    <node id="p-144">
      <data key="d0">connector</data>
    </node>
    <node id="p-145">
      <data key="d0">connector</data>
    </node>
    <node id="p-146">
      <data key="d0">connector</data>
    </node>
    <node id="p-147">
      <data key="d0">connector</data>
    </node>
    <node id="p-148">
      <data key="d0">connector</data>
    </node>
    <node id="p-149">
      <data key="d0">connector</data>
    </node>
    <node id="p-150">
      <data key="d0">connector</data>
    </node>
    <node id="p-151">
      <data key="d0">connector</data>
    </node>
    <node id="p-152">
      <data key="d0">connector</data>
    </node>
    <node id="p-153">
      <data key="d0">connector</data>
    </node>
    <node id="p-154">
      <data key="d0">connector</data>
    </node>
    <node id="p-155">
      <data key="d0">connector</data>
    </node>
    <node id="p-156">
      <data key="d0">connector</data>
    </node>
    <node id="p-157">
      <data key="d0">connector</data>
    </node>
    <node id="p-158">
      <data key="d0">connector</data>
    </node>
    <node id="p-159">
      <data key="d0">connector</data>
    </node>
    <node id="p-160">
      <data key="d0">connector</data>
    </node>
    <node id="p-161">
      <data key="d0">connector</data>
    </node>
    <node id="p-162">
      <data key="d0">connector</data>
    </node>
    <node id="p-163">
      <data key="d0">connector</data>
    </node>
    <node id="p-164">
      <data key="d0">connector</data>
    </node>
    <node id="p-165">
      <data key="d0">connector</data>
    </node>
    <node id="p-166">
      <data key="d0">connector</data>
    </node>
    <node id="p-167">
      <data key="d0">connector</data>
    </node>
    <node id="p-168">
      <data key="d0">connector</data>
    </node>
    <node id="p-169">
      <data key="d0">connector</data>
    </node>
    <node id="p-170">
      <data key="d0">connector</data>
    </node>
    <node id="p-171">
      <data key="d0">connector</data>
    </node>
    <node id="p-172">
      <data key="d0">connector</data>
    </node>
    <node id="p-173">
      <data key="d0">connector</data>
    </node>
    <node id="p-174">
      <data key="d0">connector</data>
    </node>
    <node id="p-175">
      <data key="d0">connector</data>
    </node>
    <node id="p-176">
      <data key="d0">connector</data>
    </node>
    <node id="p-177">
      <data key="d0">connector</data>
    </node>
    <node id="p-178">
      <data key="d0">connector</data>
    </node>
    <node id="p-179">
      <data key="d0">connector</data>
    </node>
    <edge source="l-0" target="l-2" />
    <edge source="l-0" target="l-3" />
    <edge source="l-0" target="p-109" />
    <edge source="l-2" target="l-9" />
    <edge source="l-2" target="l-19" />
    <edge source="l-2" target="l-22" />
    <edge source="l-2" target="l-27" />
    <edge source="l-2" target="l-30" />
    <edge source="l-2" target="l-31" />
    <edge source="l-2" target="l-40" />
    <edge source="l-2" target="l-41" />
    <edge source="l-2" target="l-50" />
    <edge source="l-2" target="l-340" />
    <edge source="l-2" target="p-16" />
    <edge source="l-2" target="l-281" />
    <edge source="l-2" target="p-47" />
    <edge source="l-2" target="l-71" />
    <edge source="l-2" target="p-44" />
    <edge source="l-3" target="l-9" />
    <edge source="l-3" target="l-19" />
    <edge source="l-3" target="l-22" />
    <edge source="l-3" target="l-24" />
    <edge source="l-3" target="l-27" />
    <edge source="l-3" target="l-30" />
    <edge source="l-3" target="l-31" />
    <edge source="l-3" target="l-40" />
    <edge source="l-3" target="l-41" />
    <edge source="l-3" target="l-50" />
    <edge source="l-3" target="l-192" />
    <edge source="l-3" target="l-198" />
    <edge source="l-3" target="p-143" />
    <edge source="l-3" target="p-39" />
    <edge source="l-3" target="p-167" />
    <edge source="l-4" target="p-7" />
    <edge source="l-4" target="p-93" />
    <edge source="l-5" target="s-88" />
    <edge source="l-5" target="p-177" />
    <edge source="l-6" target="l-552" />
    <edge source="l-6" target="s-35" />
    <edge source="l-6" target="p-131" />
    <edge source="l-6" target="s-34" />
    <edge source="l-6" target="l-231" />
    <edge source="l-8" target="s-24" />
    <edge source="l-8" target="p-25" />
    <edge source="l-8" target="l-111" />
    <edge source="l-9" target="p-56" />
    <edge source="l-9" target="p-18" />
    <edge source="l-9" target="p-70" />
    <edge source="l-10" target="p-78" />
    <edge source="l-10" target="s-66" />
    <edge source="l-12" target="p-74" />
    <edge source="l-12" target="p-103" />
    <edge source="l-12" target="p-110" />
    <edge source="l-12" target="p-127" />
    <edge source="l-12" target="p-170" />
    <edge source="l-12" target="p-176" />
    <edge source="l-12" target="s-45" />
    <edge source="l-12" target="s-32" />
    <edge source="l-13" target="p-112" />
    <edge source="l-13" target="p-148" />
    <edge source="l-14" target="s-60" />
    <edge source="l-14" target="p-12" />
    <edge source="l-14" target="p-169" />
    <edge source="l-15" target="p-11" />
    <edge source="l-15" target="p-40" />
    <edge source="l-15" target="s-80" />
    <edge source="l-17" target="s-32" />
    <edge source="l-17" target="p-71" />
    <edge source="l-17" target="p-91" />
    <edge source="l-17" target="p-137" />
    <edge source="l-17" target="s-29" />
    <edge source="l-19" target="p-73" />
    <edge source="l-19" target="p-76" />
    <edge source="l-19" target="p-87" />
    <edge source="l-19" target="p-119" />
    <edge source="l-19" target="p-173" />
    <edge source="l-21" target="s-58" />
    <edge source="l-21" target="p-52" />
    <edge source="l-22" target="p-168" />
    <edge source="l-22" target="p-171" />
    <edge source="l-22" target="p-14" />
    <edge source="l-23" target="p-126" />
    <edge source="l-23" target="s-82" />
    <edge source="l-24" target="l-156" />
    <edge source="l-24" target="p-32" />
    <edge source="l-24" target="p-92" />
    <edge source="l-24" target="p-132" />
    <edge source="l-24" target="p-151" />
    <edge source="l-24" target="p-157" />
    <edge source="l-25" target="s-39" />
    <edge source="l-25" target="p-42" />
    <edge source="l-26" target="s-43" />
    <edge source="l-26" target="s-40" />
    <edge source="l-27" target="p-14" />
    <edge source="l-27" target="p-141" />
    <edge source="l-27" target="p-171" />
    <edge source="l-28" target="s-27" />
    <edge source="l-28" target="s-2" />
    <edge source="l-29" target="s-10" />
    <edge source="l-29" target="s-12" />
    <edge source="l-29" target="s-15" />
    <edge source="l-31" target="p-18" />
    <edge source="l-31" target="p-26" />
    <edge source="l-31" target="p-70" />
    <edge source="l-31" target="p-17" />
    <edge source="l-31" target="p-133" />
    <edge source="l-33" target="s-56" />
    <edge source="l-33" target="s-57" />
    <edge source="l-33" target="s-64" />
    <edge source="l-33" target="p-82" />
    <edge source="l-35" target="p-59" />
    <edge source="l-35" target="p-96" />
    <edge source="l-35" target="s-22" />
    <edge source="l-36" target="p-122" />
    <edge source="l-36" target="s-62" />
    <edge source="l-36" target="s-68" />
    <edge source="l-39" target="s-34" />
    <edge source="l-39" target="s-33" />
    <edge source="l-40" target="p-151" />
    <edge source="l-40" target="p-92" />
    <edge source="l-40" target="p-32" />
    <edge source="l-40" target="p-132" />
    <edge source="l-40" target="p-157" />
    <edge source="l-46" target="p-20" />
    <edge source="l-46" target="p-106" />
    <edge source="l-48" target="p-66" />
    <edge source="l-48" target="p-99" />
    <edge source="l-50" target="p-14" />
    <edge source="l-50" target="p-141" />
    <edge source="l-50" target="p-171" />
    <edge source="l-54" target="p-11" />
    <edge source="l-54" target="p-126" />
    <edge source="l-56" target="p-26" />
    <edge source="l-56" target="p-56" />
    <edge source="l-58" target="s-77" />
    <edge source="l-58" target="s-79" />
    <edge source="l-58" target="s-80" />
    <edge source="l-64" target="s-29" />
    <edge source="l-64" target="p-2" />
    <edge source="l-64" target="p-131" />
    <edge source="l-65" target="p-9" />
    <edge source="l-65" target="p-83" />
    <edge source="l-65" target="s-28" />
    <edge source="l-69" target="s-3" />
    <edge source="l-69" target="s-1" />
    <edge source="l-71" target="p-23" />
    <edge source="l-71" target="p-118" />
    <edge source="l-75" target="p-20" />
    <edge source="l-75" target="p-167" />
    <edge source="l-78" target="p-145" />
    <edge source="l-78" target="p-48" />
    <edge source="l-85" target="p-51" />
    <edge source="l-85" target="p-107" />
    <edge source="l-89" target="s-36" />
    <edge source="l-89" target="p-131" />
    <edge source="l-100" target="s-48" />
    <edge source="l-100" target="p-170" />
    <edge source="l-102" target="p-128" />
    <edge source="l-102" target="p-102" />
    <edge source="l-104" target="s-13" />
    <edge source="l-104" target="s-14" />
    <edge source="l-104" target="p-35" />
    <edge source="l-104" target="p-55" />
    <edge source="l-104" target="p-158" />
    <edge source="l-105" target="p-88" />
    <edge source="l-105" target="p-170" />
    <edge source="l-105" target="s-54" />
    <edge source="l-110" target="p-116" />
    <edge source="l-110" target="p-12" />
    <edge source="l-110" target="p-152" />
    <edge source="l-111" target="s-41" />
    <edge source="l-111" target="s-30" />
    <edge source="l-114" target="s-63" />
    <edge source="l-114" target="p-29" />
    <edge source="l-116" target="p-93" />
    <edge source="l-116" target="p-79" />
    <edge source="l-116" target="p-18" />
    <edge source="l-118" target="p-4" />
    <edge source="l-118" target="p-34" />
    <edge source="l-118" target="p-45" />
    <edge source="l-118" target="p-61" />
    <edge source="l-118" target="p-121" />
    <edge source="l-118" target="p-140" />
    <edge source="l-125" target="p-32" />
    <edge source="l-125" target="p-62" />
    <edge source="l-125" target="p-111" />
    <edge source="l-125" target="p-115" />
    <edge source="l-125" target="p-173" />
    <edge source="l-128" target="s-69" />
    <edge source="l-128" target="p-67" />
    <edge source="l-128" target="s-71" />
    <edge source="l-133" target="p-49" />
    <edge source="l-133" target="p-53" />
    <edge source="l-135" target="s-49" />
    <edge source="l-135" target="p-97" />
    <edge source="l-135" target="p-5" />
    <edge source="l-136" target="l-190" />
    <edge source="l-136" target="l-363" />
    <edge source="l-136" target="s-21" />
    <edge source="l-139" target="p-159" />
    <edge source="l-139" target="p-150" />
    <edge source="l-143" target="s-64" />
    <edge source="l-143" target="p-42" />
    <edge source="l-145" target="s-79" />
    <edge source="l-145" target="s-81" />
    <edge source="l-145" target="s-78" />
    <edge source="l-148" target="p-47" />
    <edge source="l-148" target="p-39" />
    <edge source="l-152" target="s-70" />
    <edge source="l-152" target="p-177" />
    <edge source="l-156" target="l-340" />
    <edge source="l-160" target="s-38" />
    <edge source="l-160" target="s-37" />
    <edge source="l-163" target="p-106" />
    <edge source="l-163" target="p-19" />
    <edge source="l-163" target="p-73" />
    <edge source="l-168" target="s-81" />
    <edge source="l-168" target="s-82" />
    <edge source="l-170" target="p-23" />
    <edge source="l-170" target="p-44" />
    <edge source="l-170" target="p-76" />
    <edge source="l-172" target="s-76" />
    <edge source="l-172" target="p-177" />
    <edge source="l-175" target="p-1" />
    <edge source="l-175" target="p-27" />
    <edge source="l-175" target="p-146" />
    <edge source="l-183" target="s-63" />
    <edge source="l-183" target="p-12" />
    <edge source="l-190" target="p-160" />
    <edge source="l-190" target="s-25" />
    <edge source="l-192" target="p-21" />
    <edge source="l-192" target="p-155" />
    <edge source="l-192" target="p-62" />
    <edge source="l-195" target="l-512" />
    <edge source="l-195" target="s-39" />
    <edge source="l-195" target="s-43" />
    <edge source="l-196" target="p-66" />
    <edge source="l-196" target="p-112" />
    <edge source="l-198" target="p-8" />
    <edge source="l-198" target="p-45" />
    <edge source="l-198" target="p-140" />
    <edge source="l-200" target="p-4" />
    <edge source="l-200" target="p-61" />
    <edge source="l-200" target="p-115" />
    <edge source="l-200" target="p-165" />
    <edge source="l-206" target="s-67" />
    <edge source="l-206" target="p-122" />
    <edge source="l-206" target="p-29" />
    <edge source="l-211" target="s-33" />
    <edge source="l-211" target="p-138" />
    <edge source="l-211" target="s-26" />
    <edge source="l-215" target="s-14" />
    <edge source="l-215" target="p-63" />
    <edge source="l-218" target="p-116" />
    <edge source="l-218" target="p-78" />
    <edge source="l-222" target="s-4" />
    <edge source="l-222" target="p-85" />
    <edge source="l-229" target="p-84" />
    <edge source="l-229" target="p-169" />
    <edge source="l-229" target="p-52" />
    <edge source="l-230" target="s-5" />
    <edge source="l-230" target="s-0" />
    <edge source="l-231" target="s-4" />
    <edge source="l-231" target="p-31" />
    <edge source="l-236" target="p-98" />
    <edge source="l-236" target="p-49" />
    <edge source="l-238" target="p-22" />
    <edge source="l-238" target="p-44" />
    <edge source="l-238" target="p-62" />
    <edge source="l-241" target="p-65" />
    <edge source="l-241" target="p-88" />
    <edge source="l-245" target="p-176" />
    <edge source="l-245" target="s-47" />
    <edge source="l-248" target="s-9" />
    <edge source="l-248" target="s-7" />
    <edge source="l-252" target="s-55" />
    <edge source="l-252" target="p-82" />
    <edge source="l-256" target="p-115" />
    <edge source="l-256" target="p-123" />
    <edge source="l-256" target="l-281" />
    <edge source="l-264" target="p-169" />
    <edge source="l-264" target="p-126" />
    <edge source="l-266" target="s-1" />
    <edge source="l-266" target="p-85" />
    <edge source="l-267" target="s-5" />
    <edge source="l-267" target="s-3" />
    <edge source="l-268" target="p-175" />
    <edge source="l-268" target="s-60" />
    <edge source="l-271" target="s-89" />
    <edge source="l-271" target="s-88" />
    <edge source="l-272" target="p-123" />
    <edge source="l-272" target="p-157" />
    <edge source="l-275" target="s-46" />
    <edge source="l-275" target="s-40" />
    <edge source="l-276" target="p-42" />
    <edge source="l-276" target="s-62" />
    <edge source="l-278" target="p-27" />
    <edge source="l-278" target="p-90" />
    <edge source="l-278" target="p-146" />
    <edge source="l-278" target="p-75" />
    <edge source="l-279" target="p-104" />
    <edge source="l-279" target="p-70" />
    <edge source="l-279" target="p-79" />
    <edge source="l-281" target="p-144" />
    <edge source="l-287" target="s-67" />
    <edge source="l-287" target="p-52" />
    <edge source="l-289" target="p-34" />
    <edge source="l-289" target="p-61" />
    <edge source="l-289" target="p-125" />
    <edge source="l-289" target="p-155" />
    <edge source="l-289" target="p-165" />
    <edge source="l-294" target="p-160" />
    <edge source="l-294" target="p-59" />
    <edge source="l-298" target="s-35" />
    <edge source="l-298" target="p-138" />
    <edge source="l-310" target="p-19" />
    <edge source="l-310" target="p-47" />
    <edge source="l-310" target="p-76" />
    <edge source="l-316" target="p-17" />
    <edge source="l-316" target="p-38" />
    <edge source="l-316" target="p-57" />
    <edge source="l-324" target="s-78" />
    <edge source="l-324" target="s-61" />
    <edge source="l-326" target="s-18" />
    <edge source="l-326" target="p-25" />
    <edge source="l-332" target="s-65" />
    <edge source="l-332" target="p-122" />
    <edge source="l-335" target="s-8" />
    <edge source="l-335" target="p-85" />
    <edge source="l-336" target="p-6" />
    <edge source="l-336" target="p-37" />
    <edge source="l-336" target="p-139" />
    <edge source="l-336" target="p-156" />
    <edge source="l-336" target="p-105" />
    <edge source="l-338" target="p-41" />
    <edge source="l-338" target="p-51" />
    <edge source="l-338" target="p-107" />
    <edge source="l-342" target="s-74" />
    <edge source="l-342" target="s-72" />
    <edge source="l-344" target="s-44" />
    <edge source="l-344" target="s-37" />
    <edge source="l-347" target="p-105" />
    <edge source="l-347" target="p-125" />
    <edge source="l-351" target="p-84" />
    <edge source="l-351" target="s-55" />
    <edge source="l-352" target="s-24" />
    <edge source="l-352" target="p-160" />
    <edge source="l-354" target="s-52" />
    <edge source="l-354" target="p-65" />
    <edge source="l-356" target="s-22" />
    <edge source="l-356" target="p-179" />
    <edge source="l-359" target="p-23" />
    <edge source="l-359" target="p-144" />
    <edge source="l-360" target="p-136" />
    <edge source="l-360" target="p-142" />
    <edge source="l-363" target="s-20" />
    <edge source="l-363" target="s-38" />
    <edge source="l-369" target="s-53" />
    <edge source="l-369" target="p-97" />
    <edge source="l-370" target="p-16" />
    <edge source="l-370" target="p-38" />
    <edge source="l-371" target="s-58" />
    <edge source="l-371" target="p-82" />
    <edge source="l-378" target="p-3" />
    <edge source="l-378" target="p-104" />
    <edge source="l-378" target="p-124" />
    <edge source="l-385" target="p-13" />
    <edge source="l-385" target="p-86" />
    <edge source="l-386" target="p-41" />
    <edge source="l-386" target="p-99" />
    <edge source="l-389" target="p-4" />
    <edge source="l-389" target="p-34" />
    <edge source="l-389" target="p-92" />
    <edge source="l-389" target="p-165" />
    <edge source="l-394" target="p-138" />
    <edge source="l-394" target="p-71" />
    <edge source="l-399" target="p-65" />
    <edge source="l-399" target="s-47" />
    <edge source="l-407" target="s-26" />
    <edge source="l-407" target="p-137" />
    <edge source="l-408" target="p-30" />
    <edge source="l-408" target="p-149" />
    <edge source="l-408" target="p-159" />
    <edge source="l-429" target="p-43" />
    <edge source="l-429" target="p-54" />
    <edge source="l-429" target="p-64" />
    <edge source="l-429" target="p-118" />
    <edge source="l-429" target="p-154" />
    <edge source="l-437" target="p-0" />
    <edge source="l-437" target="p-129" />
    <edge source="l-437" target="p-163" />
    <edge source="l-444" target="p-49" />
    <edge source="l-444" target="p-53" />
    <edge source="l-445" target="s-85" />
    <edge source="l-445" target="s-83" />
    <edge source="l-449" target="s-6" />
    <edge source="l-449" target="p-91" />
    <edge source="l-459" target="s-41" />
    <edge source="l-459" target="s-42" />
    <edge source="l-464" target="p-36" />
    <edge source="l-464" target="p-142" />
    <edge source="l-465" target="p-179" />
    <edge source="l-465" target="s-10" />
    <edge source="l-467" target="p-36" />
    <edge source="l-467" target="p-0" />
    <edge source="l-470" target="s-20" />
    <edge source="l-470" target="p-25" />
    <edge source="l-474" target="s-11" />
    <edge source="l-474" target="p-35" />
    <edge source="l-474" target="p-55" />
    <edge source="l-474" target="p-58" />
    <edge source="l-479" target="p-87" />
    <edge source="l-479" target="p-155" />
    <edge source="l-480" target="p-1" />
    <edge source="l-480" target="p-27" />
    <edge source="l-480" target="p-90" />
    <edge source="l-480" target="p-145" />
    <edge source="l-487" target="s-87" />
    <edge source="l-487" target="p-40" />
    <edge source="l-494" target="p-8" />
    <edge source="l-494" target="p-45" />
    <edge source="l-494" target="p-121" />
    <edge source="l-494" target="p-151" />
    <edge source="l-497" target="s-72" />
    <edge source="l-497" target="p-5" />
    <edge source="l-499" target="p-22" />
    <edge source="l-499" target="p-73" />
    <edge source="l-500" target="p-67" />
    <edge source="l-500" target="s-66" />
    <edge source="l-511" target="p-43" />
    <edge source="l-511" target="p-54" />
    <edge source="l-511" target="p-101" />
    <edge source="l-511" target="p-117" />
    <edge source="l-511" target="p-123" />
    <edge source="l-512" target="p-74" />
    <edge source="l-512" target="p-127" />
    <edge source="l-512" target="p-134" />
    <edge source="l-516" target="p-136" />
    <edge source="l-516" target="p-135" />
    <edge source="l-520" target="s-25" />
    <edge source="l-520" target="s-23" />
    <edge source="l-522" target="s-17" />
    <edge source="l-522" target="p-96" />
    <edge source="l-529" target="p-30" />
    <edge source="l-529" target="p-81" />
    <edge source="l-529" target="p-153" />
    <edge source="l-529" target="p-162" />
    <edge source="l-536" target="p-6" />
    <edge source="l-536" target="p-8" />
    <edge source="l-536" target="p-100" />
    <edge source="l-536" target="p-121" />
    <edge source="l-536" target="p-139" />
    <edge source="l-536" target="p-140" />
    <edge source="l-541" target="p-91" />
    <edge source="l-541" target="s-27" />
    <edge source="l-542" target="p-30" />
    <edge source="l-542" target="p-147" />
    <edge source="l-542" target="p-149" />
    <edge source="l-542" target="p-153" />
    <edge source="l-542" target="p-162" />
    <edge source="l-545" target="p-163" />
    <edge source="l-545" target="p-60" />
    <edge source="l-552" target="p-10" />
    <edge source="l-552" target="p-24" />
    <edge source="l-564" target="s-19" />
    <edge source="l-564" target="p-174" />
    <edge source="l-566" target="p-46" />
    <edge source="l-566" target="p-143" />
    <edge source="l-573" target="p-86" />
    <edge source="l-573" target="p-48" />
    <edge source="l-581" target="p-152" />
    <edge source="l-581" target="s-71" />
    <edge source="l-582" target="s-23" />
    <edge source="l-582" target="s-17" />
    <edge source="l-585" target="p-97" />
    <edge source="l-585" target="s-46" />
    <edge source="l-591" target="p-84" />
    <edge source="l-591" target="p-29" />
    <edge source="l-594" target="s-50" />
    <edge source="l-594" target="p-103" />
    <edge source="l-597" target="p-15" />
    <edge source="l-597" target="p-64" />
    <edge source="l-599" target="p-161" />
    <edge source="l-599" target="p-143" />
    <edge source="l-601" target="p-95" />
    <edge source="l-601" target="p-133" />
    <edge source="l-604" target="s-31" />
    <edge source="l-604" target="s-45" />
    <edge source="l-615" target="p-56" />
    <edge source="l-615" target="p-168" />
    <edge source="l-619" target="p-63" />
    <edge source="l-619" target="s-16" />
    <edge source="l-621" target="p-94" />
    <edge source="l-621" target="s-61" />
    <edge source="l-637" target="p-20" />
    <edge source="l-637" target="p-87" />
    <edge source="l-648" target="p-55" />
    <edge source="l-648" target="p-58" />
    <edge source="l-648" target="p-158" />
    <edge source="l-648" target="s-15" />
    <edge source="l-666" target="p-119" />
    <edge source="l-666" target="p-167" />
    <edge source="l-686" target="s-84" />
    <edge source="l-686" target="s-83" />
    <edge source="l-695" target="p-141" />
    <edge source="l-695" target="p-168" />
    <edge source="l-700" target="p-13" />
    <edge source="l-700" target="p-98" />
    <edge source="l-701" target="p-66" />
    <edge source="l-701" target="p-112" />
    <edge source="l-704" target="p-28" />
    <edge source="l-704" target="p-50" />
    <edge source="l-704" target="p-89" />
    <edge source="l-707" target="p-67" />
    <edge source="l-707" target="s-73" />
    <edge source="l-716" target="p-60" />
    <edge source="l-716" target="p-75" />
    <edge source="l-728" target="p-120" />
    <edge source="l-728" target="p-107" />
    <edge source="l-738" target="p-15" />
    <edge source="l-738" target="p-111" />
    <edge source="l-740" target="p-71" />
    <edge source="l-740" target="p-9" />
    <edge source="l-741" target="p-78" />
    <edge source="l-741" target="s-68" />
    <edge source="l-747" target="p-3" />
    <edge source="l-747" target="p-46" />
    <edge source="l-747" target="p-124" />
    <edge source="l-748" target="p-68" />
    <edge source="l-748" target="p-164" />
    <edge source="l-748" target="p-172" />
    <edge source="l-748" target="s-52" />
    <edge source="l-752" target="s-21" />
    <edge source="l-752" target="p-179" />
    <edge source="l-760" target="p-110" />
    <edge source="l-760" target="p-127" />
    <edge source="l-760" target="p-134" />
    <edge source="l-760" target="p-178" />
    <edge source="l-766" target="p-106" />
    <edge source="l-766" target="p-173" />
    <edge source="l-772" target="p-69" />
    <edge source="l-772" target="p-135" />
    <edge source="l-780" target="p-113" />
    <edge source="l-780" target="p-94" />
    <edge source="l-782" target="p-11" />
    <edge source="l-782" target="s-85" />
    <edge source="l-803" target="p-68" />
    <edge source="l-803" target="p-80" />
    <edge source="l-803" target="p-172" />
    <edge source="l-834" target="s-51" />
    <edge source="l-834" target="p-176" />
    <edge source="l-840" target="p-150" />
    <edge source="l-840" target="p-161" />
    <edge source="l-850" target="p-63" />
    <edge source="l-850" target="p-59" />
    <edge source="l-852" target="p-40" />
    <edge source="l-852" target="s-86" />
    <edge source="l-855" target="p-137" />
    <edge source="l-855" target="s-28" />
    <edge source="l-859" target="p-28" />
    <edge source="l-859" target="p-89" />
    <edge source="l-866" target="p-43" />
    <edge source="l-866" target="p-101" />
    <edge source="l-866" target="p-117" />
    <edge source="l-866" target="p-154" />
    <edge source="l-884" target="p-60" />
    <edge source="l-884" target="p-75" />
    <edge source="l-885" target="p-13" />
    <edge source="l-885" target="p-86" />
    <edge source="l-885" target="p-98" />
    <edge source="l-891" target="p-6" />
    <edge source="l-891" target="p-21" />
    <edge source="l-891" target="p-37" />
    <edge source="l-891" target="p-100" />
    <edge source="l-900" target="p-136" />
    <edge source="l-900" target="p-142" />
    <edge source="l-905" target="p-149" />
    <edge source="l-905" target="p-159" />
    <edge source="l-913" target="p-72" />
    <edge source="l-913" target="p-120" />
    <edge source="l-923" target="p-41" />
    <edge source="l-923" target="p-51" />
    <edge source="l-923" target="p-99" />
    <edge source="l-932" target="p-114" />
    <edge source="l-932" target="p-133" />
    <edge source="l-940" target="s-0" />
    <edge source="l-940" target="p-77" />
    <edge source="l-949" target="p-1" />
    <edge source="l-949" target="p-90" />
    <edge source="l-949" target="p-145" />
    <edge source="l-949" target="p-146" />
    <edge source="l-953" target="p-7" />
    <edge source="l-953" target="p-93" />
    <edge source="l-953" target="p-108" />
    <edge source="l-955" target="p-17" />
    <edge source="l-955" target="p-57" />
    <edge source="l-955" target="p-114" />
    <edge source="l-963" target="p-0" />
    <edge source="l-963" target="p-129" />
    <edge source="l-964" target="p-125" />
    <edge source="l-964" target="p-111" />
    <edge source="l-977" target="p-21" />
    <edge source="l-977" target="p-119" />
    <edge source="l-989" target="p-96" />
    <edge source="l-989" target="s-16" />
    <edge source="l-990" target="p-81" />
    <edge source="l-990" target="p-89" />
    <edge source="l-990" target="p-147" />
    <edge source="l-990" target="p-162" />
    <edge source="l-998" target="p-132" />
    <edge source="l-998" target="p-144" />
    <edge source="l-1003" target="p-22" />
    <edge source="l-1003" target="p-101" />
    <edge source="l-1006" target="p-102" />
    <edge source="l-1006" target="p-50" />
    <edge source="l-1028" target="p-33" />
    <edge source="l-1028" target="p-95" />
    <edge source="l-1042" target="p-9" />
    <edge source="l-1042" target="p-130" />
    <edge source="l-1046" target="p-7" />
    <edge source="l-1046" target="p-33" />
    <edge source="l-1046" target="p-108" />
    <edge source="l-1047" target="p-10" />
    <edge source="l-1047" target="p-24" />
    <edge source="l-1063" target="p-79" />
    <edge source="l-1063" target="p-26" />
    <edge source="l-1070" target="p-69" />
    <edge source="l-1070" target="p-135" />
    <edge source="l-1071" target="p-31" />
    <edge source="l-1071" target="p-166" />
    <edge source="l-1078" target="p-80" />
    <edge source="l-1078" target="p-164" />
    <edge source="l-1078" target="p-172" />
    <edge source="l-1082" target="p-15" />
    <edge source="l-1082" target="p-64" />
    <edge source="l-1083" target="p-46" />
    <edge source="l-1083" target="p-124" />
    <edge source="l-1092" target="s-87" />
    <edge source="l-1092" target="s-84" />
    <edge source="l-1099" target="p-3" />
    <edge source="l-1099" target="p-104" />
    <edge source="l-1105" target="p-16" />
    <edge source="l-1105" target="p-38" />
    <edge source="l-1106" target="p-33" />
    <edge source="l-1106" target="p-95" />
    <edge source="l-1106" target="p-108" />
    <edge source="l-1108" target="p-35" />
    <edge source="l-1108" target="p-58" />
    <edge source="l-1108" target="p-158" />
    <edge source="l-1116" target="p-72" />
    <edge source="l-1116" target="p-120" />
    <edge source="l-1119" target="s-49" />
    <edge source="l-1119" target="p-103" />
    <edge source="l-1122" target="p-37" />
    <edge source="l-1122" target="p-100" />
    <edge source="l-1122" target="p-139" />
    <edge source="l-1122" target="p-156" />
    <edge source="l-1123" target="p-54" />
    <edge source="l-1123" target="p-117" />
    <edge source="l-1123" target="p-118" />
    <edge source="l-1123" target="p-154" />
    <edge source="l-1124" target="p-174" />
    <edge source="l-1124" target="s-18" />
    <edge source="l-1128" target="p-81" />
    <edge source="l-1128" target="p-147" />
    <edge source="l-1128" target="p-153" />
    <edge source="l-1132" target="p-28" />
    <edge source="l-1132" target="p-50" />
    <edge source="l-1136" target="p-129" />
    <edge source="l-1136" target="p-163" />
    <edge source="l-1137" target="p-57" />
    <edge source="l-1137" target="p-114" />
    <edge source="l-1138" target="s-30" />
    <edge source="l-1138" target="p-130" />
    <edge source="l-1139" target="p-74" />
    <edge source="l-1139" target="p-110" />
    <edge source="l-1139" target="p-134" />
    <edge source="l-1141" target="p-150" />
    <edge source="l-1141" target="p-161" />
    <edge source="s-9" target="p-24" />
    <edge source="s-42" target="p-178" />
    <edge source="s-59" target="p-113" />
    <edge source="s-59" target="p-175" />
    <edge source="s-61" target="p-94" />
    <edge source="s-75" target="p-68" />
    <edge source="s-75" target="p-80" />
    <edge source="s-75" target="p-164" />
  </graph>
</graphml>
  """
)

plt.clf()
plt.figure(figsize=(100, 80))
pos = nx.spring_layout(r_graph, k=2)  
nx.draw(r_graph,  with_labels=True, node_size=100, node_color="lightgreen", font_size=10, font_weight="bold")
plt.title("Undirected Graph")
plt.show()

In [ ]:
len(graph_service.graph['l-0'])

In [ ]:
graph_service.generate_graphml()

In [ ]:
import networkx as nx


# this can be used for utility purposes.

def subgraph_between_nodes(G, start_node, end_node):
    # Find a simple path between the two nodes
    paths = list(nx.all_simple_paths(G, source=start_node, target=end_node))
    
    # Create a subgraph from the first valid path
    if paths:
        path = paths[0]
        subgraph = G.subgraph(path).copy()
        
        # Remove any edges that create duplicate connections
        for node in path:
            neighbors = list(subgraph.neighbors(node))
            # Only keep the neighbor that is part of the path
            for neighbor in neighbors:
                if neighbor not in path:
                    subgraph.remove_edge(node, neighbor)
                    
        return subgraph
    else:
        return None


service.display_graph(
    subgraph_between_nodes(graph_service.graph, "l-16", "l-6")
)